# Test

In [2]:
#!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

import pandas as pd
import numpy as np

import os
import sys
import shutil

import cv2
from mtcnn import MTCNN

from tqdm.notebook import tqdm
import random

import warnings
warnings.filterwarnings("ignore")

Num GPUs Available:  1


In [3]:
train_videos_path = '../input/train_videos/'
train_metadata_path = '../input/train_metadata/'
train_images_path = "../input/test_images/" # path to save train images to

In [4]:
train_videos_files = [] # List of all train videos paths
train_metadata_files = [] # List of train metadata paths

for folder in enumerate(os.listdir(train_videos_path)):
    for file in os.listdir(train_videos_path + folder[1]):
        if file == 'metadata.json':
            # Rename and copy the metadata to a new directory
            old_path = train_videos_path + folder[1] + '/' + file
            new_path = train_metadata_path + 'metadata' + str(folder[0]) + '.json'
            shutil.copy(old_path, new_path)            
            train_metadata_files.append(new_path)
        else:
            train_videos_files.append(train_videos_path + folder[1] + '/'+ file)

In [5]:
def extract_faces(videos_dir_path, images_dir_path, frames=1, conf_level=0.95):
    """
    Inputs a directory of videos, extracts n frames. 
    Outputs images of ANY faces detected in those frames.
    
    videos_dir_path: (str) Path to your directory of videos
    images_dir_path: (str) Path to where you'll save your images to
    frames: (int or list) Number of frames. If int, take that many frames. If list, take frame numbers specified in list. 
    conf_level: (float) Confidence level for the face recognition model.
    """
    def crop(img, x, y, w, h):
        """
        Crop and reshape images to be uniform across all frames
        """
        x -= 40
        y -= 40
        w += 80
        h += 80
        if x < 0:
            x = 0
        if y <= 0:
            y = 0
        return cv2.cvtColor(cv2.resize(img[y:y + h, x:x + w], (256, 256)), cv2.COLOR_BGR2RGB)
    
    if type(videos_dir_path) == list: 
        videos_dir = videos_dir_path
    else: 
        videos_dir = os.listdir(videos_dir_path) # List train vids
    
    # Extract images from videos
    if type(frames) == list:
        print(f'Extracting frames {frames} from videos')
    else:
        print(f'Extracting {frames} random frame(s) from videos')
        
    with tqdm(total=len(range(0, 3))) as pbar: #len(videos_dir)
        for i in range(0, 3): #len(videos_dir)
            try:
                if type(videos_dir_path) == list: 
                    file_name = videos_dir_path[i].split('/')[4]
                    file_path = videos_dir_path[i]
                    vid_name = file_name.split('.')[0]
                else: 
                    file_name = videos_dir[i] # file name with .ext
                    file_path = videos_dir_path + file_name # full file path
                    vid_name = file_name.split('.')[0] # file name without .ext

                if type(frames) == list:
                    for num in range(0, len(frames)):
                        cap = cv2.VideoCapture(file_path)
                        total_frames = cap.get(7)
                        vid_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                        cap.set(1, num) # EDIT HERE FOR FRAME NUMBER
                        ret, frame = cap.read()
                        image_name = vid_name + '_' + str(num) + '.jpg'
                        cv2.imwrite(os.path.join(train_images_path, image_name), frame) # Save frame as image
                        cv2.destroyAllWindows()
                        cap.release()
                else:
                    for num in range(0, frames):
                        cap = cv2.VideoCapture(file_path)
                        total_frames = cap.get(7)
                        vid_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                        cap.set(1, random.randint(0, vid_length)) # EDIT HERE FOR FRAME NUMBER
                        ret, frame = cap.read()
                        image_name = vid_name + '_' + str(num) + '.jpg'
                        cv2.imwrite(os.path.join(train_images_path, image_name), frame) # Save frame as image
                        cv2.destroyAllWindows()
                        cap.release()
                pbar.update(1)
            except:
                pass
    images_dir = os.listdir(images_dir_path) # List newly created training images
    detector = MTCNN()

    print('Extracting faces from frames')
    with tqdm(total=len(images_dir)) as pbar:
        
        for image in range(0, len(images_dir)):
            try:
                image_name = images_dir[image].split('.')[0] # Get image name without .ext
             # Read image and detect faces
                frame = cv2.imread(images_dir_path + images_dir[image])
                result = detector.detect_faces(frame)
             # Extract and save faces as their own images
                for face in range(0, len(result)):
                    # Only extract the face if confidence is more than or equal to default 0.95
                    if result[face]['confidence'] >= conf_level:            
                        startX, startY, endX, endY = result[face]['box'] # Get box coordinates
                        crop_img = crop(frame, startX, startY, endX, endY)
                        cv2.imwrite(images_dir_path + image_name + '_' + str(face) + '.jpg', crop_img)
                os.remove(images_dir_path + images_dir[image]) # Delete original images
            except:
                pass
            pbar.update(1)

In [6]:
extract_faces(train_videos_files, train_images_path, frames=2)

Extracting 2 random frame(s) from videos


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as k

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import Callback, EarlyStopping

#tf.debugging.set_log_device_placement(True) # Enable GPU logging
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1,1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1,1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
    # defining name basis
    conv_name_base='res' + str(stage) + block + '_branch'
    bn_name_base='bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides=(s,s), name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides=(s,s), name = conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes=2):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape=(64, 64, 3), classes=2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.load_weights('../output/resnet50_0.h5')